In [1]:
import sqlalchemy as sa
import os
import sys
import urllib.parse as url
import pandas as pd

In [2]:
username = os.getlogin()
external_folder_path = 'C:/Users/' + username + '/Documents/Projects/Financial_Securities/Custom_Python_Functions/'
sys.path.append(external_folder_path)
from custom_python_functions import create_connection, load_key, decrypt

key1 = 'user_key.ky'
key_file1 = 'user_key.txt'
key2 = 'pass_key.ky'
key_file2 = 'pass_key.txt'

key1 = load_key(external_folder_path, key1)
uid = decrypt(external_folder_path, key_file1, key1)

key2 = load_key(external_folder_path, key2)
passwd = decrypt(external_folder_path, key_file2, key2)

# Setup connection parameters
server = 'danvuk.database.windows.net'
dbase = 'Financial_Securities'

# Create a connection to the database
s, e = create_connection(server, dbase, uid, passwd)
s1 = s()  # Instantiate a session object

In [3]:
Base = sa.orm.declarative_base()

class Equities(Base):
    
    """
    SQLAlchemy ORM class representing the 'Equities' table in the 'Equities' schema.

    Attributes:
    __tablename__ (str): The name of the table in the database.
    __table_args__ (dict): Additional arguments for the table, including schema name.
    Ticker_ID (Column): A unique identifier for each equity; part of the primary key.
    Ticker (Column): The ticker symbol of the equity.
    Name (Column): The name of the equity.
    Sub_Industry_ID (Column): The Sub-Industry_ID of the ticker.
    """
    
    __tablename__ = 'Equities'
    __table_args__ = {'schema': 'Equities'}
    Ticker_ID = sa.Column(sa.Integer, primary_key=True)
    Ticker = sa.Column(sa.String)
    Name = sa.Column(sa.String)
    Sub_Industry_ID = sa.Column(sa.Integer)

In [4]:
# SQL query to select and trim description fields from the Data_STG table
sql_stat = """SELECT 
               TRIM(Description) AS Description, 
               TRIM(Description2) AS Description2,
               Int_Value1
              FROM [Financial_Securities].[Equities].[Data_STG]
              ORDER BY Description"""

try:              
    df_equities = pd.read_sql(sql_stat, s1.bind)   # Execute the SQL query through the session and bind the data to the df_equities dataframe

    
# Handle SQLAlchemy errors if they occur during query execution
except sa.exc.SQLAlchemyError as e:
        # Print the error
        print(f"Issue querying Data_STG database table for count! Error: {e}")
        s1.close()  # Close the session
        raise  # Re-raise the exception to propagate the error

In [5]:
for index, row in df_equities.iterrows():
    try:
        
        # Query the 'Equities' table to find records where the 'Ticker' column matches the value in the DataFrame's 'Description' column
        q1 = s1.query(Equities).filter(Equities.Ticker == row.Description)

        # Check if any records were found with the specified 'Ticker'
        if (q1.count() >= 1):
            # If one or more records are found, get the first matching record
            q1 = s1.query(Equities).filter(Equities.Ticker == row.Description).first()
            # Update the 'Name' attribute of the found record with the value from the DataFrame's 'Description2' column
            q1.Name=row['Description2']
            # Update the 'Sub_Industry_ID' attribute of the found record with the value from the DataFrame's 'Int_Value1' column
            q1.Sub_Industry_ID=row['Int_Value1']
            
        else:
            
            # Create a new Equities object for each row in df_equities dataframe
            q1 = Equities(
                Ticker=row['Description'],
                Name=row['Description2'],
                Sub_Industry_ID=row['Int_Value1']
            )
            s1.add(q1)  # Add the instance to the session
    
     # Handle SQLAlchemy errors if they occur during adding the object
    except sa.exc.SQLAlchemyError as e:
        message = f"Issue with updating Equities database table for Ticker: {row.Description}. Error: {e}"
        print(message)
        s1.close()  # Close the session
        raise  # Re-raise the exception to propagate the error
        
s1.commit()  # Commit the transactions to the database

print("Database data load is complete")


Database data load is complete


In [6]:
# SQL query to count the number of records in the Data_STG table
sql_stat2 = """SELECT COUNT(*) FROM [Financial_Securities].[Equities].[Data_STG]"""
          
try: 
    result1 = e.execute(sql_stat2)  # Execute the count query
    cnt_recs1 = result1.scalar()  # Get the count of records
    
# Handle SQLAlchemy errors if they occur during query execution
except sa.exc.SQLAlchemyError as e:
    print(f"Issue querying Data_STG database table for count! Error: {e}")


# SQL query to count the number of records in the Equities table
sql_stat3 = """SELECT COUNT(*) FROM [Financial_Securities].[Equities].[Equities]"""
              
try: 
    result1 = e.execute(sql_stat3)  # Execute the count query
    cnt_recs2 = result1.scalar()  # Get the count of records
    
# Handle SQLAlchemy errors if they occur during query execution
except sa.exc.SQLAlchemyError as e:   
    print(f"Issue querying Sub_Industries database table for count! Error: {e}")

# Compare the record counts and print the result
if cnt_recs2 < cnt_recs1:
    print(f"Only {cnt_recs2} of {cnt_recs1} records were loaded into Equities database table!")
else:
    print(f"All {cnt_recs2} records were loaded into Equities database table!") 

All 503 records were loaded into Equities database table!


In [7]:
s1.close()  # Close the session